In [5]:
import snappy
import snappyConfig, snappyOperators as sp
from os.path import join
from glob import iglob
import numpy as np
import subprocess
import jpy
from multiprocessing import Process, Pool
from snappy import WKTReader
from snappy import HashMap
from snappy import GPF
import matplotlib.pyplot as plt
import shapefile
import pygeoif

shp_file_path = "E:\Sentinel 1 SAR Processing\Sentinel_Mosaic\Data\Spatial Data\NPL_adm0.shp"
shp_file = shapefile.Reader(shp_file_path)

g = []

for s in shp_file.shapes():
    g.append(pygeoif.geometry.as_shape(s))
    
m = pygeoif.MultiPoint(g)

wkt = str(m.wkt).replace("MULTIPOINT", "POLYGON(") + ")"

geometry = WKTReader().read(wkt)

In [9]:
processing_output_dir = r"E:\Sentinel 1 SAR Processing\Sentinel_Mosaic\Processing"
output_dir = r"E:\Sentinel 1 SAR Processing\Sentinel_Mosaic\Temp"

Product_PATH = r"E:\Sentinel 1 SAR Processing\Sentinel_Mosaic\Data"
S1_input_files = sorted(list(iglob(join(Product_PATH, 'S1*.zip'))))

# Projection
UTM_WGS84 = "GEOGCS[\"WGS84(DD)\",DATUM[\"WGS84\",SPHEROID[\"WGS84\", 6378137.0, 298.257223563]]," \
            "PRIMEM[\"Greenwich\", 0.0],UNIT[\"degree\", 0.017453292519943295],AXIS[\"Geodetic longitude\", EAST]," \
            "AXIS[\"Geodetic latitude\", NORTH]] " 


output_dir = r"E:\Sentinel 1 SAR Processing\Sentinel_Mosaic\Output"

def S1_moasic_preprocessing(inFile):
    # Raw File
    #inFile = S1_input_files[0]

    # Read File
    read_out = sp.read(inFile)
    print("Read Complete: {}".format(read_out.getName()))
    # read_out.getName())

    # Apply Orbit File
    orb_out = sp.ApplyOrbitFile(read_out)
    print("Apply Orbit File Complete")
    # orb_out.getName())

    # Radiometric Calibration
    cal_out = sp.RadiometricCalibration(orb_out)
    print("Calibration Complete")
    #print(cal_out.getName())

    # Multilook
    mul_out = sp.Multilook(cal_out)
    #print(mul_out.getName())
    print("Multilook Complete")

    # Subset
    parameters = HashMap()
    parameters.put("copyMetadata", True)
    parameters.put("GeoRegion", geometry)
    sub_out = GPF.createProduct("Subset", parameters, mul_out)
    print("Finished Subsetting")
    
    # Terrain Correction
    TC_out = sp.TerrainCorrection(sub_out, UTM_WGS84)
    #print(TC_out.getName())
    print("Terrain Correction Complete")
    return TC_out
    # Write 
    #print("Writing Terrain Corrected Output")
    #sp.write(TC_out, output_dir + "/" + TC_out.getName())  
    #print("Finished Writing Terrain Corrected Product: {}".format(TC_out.getName()))

In [10]:
prd = S1_moasic_preprocessing(S1_input_files[0])

Read Complete: S1A_IW_GRDH_1SDV_20200111T000302_20200111T000327_030745_038671_1CC4
Apply Orbit File Complete
Calibration Complete
Multilook Complete
Finished Subsetting
Terrain Correction Complete


In [6]:
def plotBand(product, band, vmin, vmax):
    band = product.getBand(band)
    w = band.getRasterWidth()
    h = band.getRasterHeight()
    print(w, h)
    band_data = np.zeros(w * h, np.float32)
    band.readPixels(0, 0, w, h, band_data)
    band_data.shape = h, w
    width = 12
    height = 12
    plt.figure(figsize=(width, height))
    imgplot = plt.imshow(band_data, cmap=plt.cm.binary, vmin=vmin, vmax=vmax)
    return imgplot

In [13]:
list(prd.getBandNames())

['Sigma0_VH', 'Sigma0_VV']

In [ ]:
plotBand(prd, "Sigma0_VV", 0, 750)